In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.image as mp
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
with open('../input/en2deformultimodelmac/training/train.de') as f:
    train_de = f.read().split('\n')
with open('../input/en2deformultimodelmac/training/train.en') as f:
    train_en = f.read().split('\n')
with open('../input/en2deformultimodelmac/splits/train_images.txt') as f:
    train_img_name = f.read().split('\n')
train_de.pop()
train_de.pop()
train_en.pop()
train_en.pop()
train_img_name.pop()
print(len(train_de))
print(len(train_en))
print(len(train_img_name))
img_path=[]
for s in train_img_name:
    img_path.append("../input/flickr-image-dataset/flickr30k_images/flickr30k_images/"+s)

In [ ]:
im=mp.imread(img_path[1])
plt.imshow(im)
print(img_path[1])
print("de:"+train_de[1])
print("en:"+train_en[1])

In [ ]:
choicenum=5000
train_de=train_de[:choicenum]
train_en=train_en[:choicenum]

In [ ]:
de_df = pd.DataFrame(train_de, columns=['de']) 
en_df = pd.DataFrame(train_en, columns=['en']) 
import re
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''
    text = text.lower()
    text = re.sub(r" ᠃", "", text)
    text = re.sub(r" ᠂", "", text)
    text = re.sub(r"-", " ", text)
    text = re.sub(r"<5>", "5", text)
    text = re.sub(r"“ ", "", text)
    text = re.sub(r" ”", "", text)
    text = re.sub(r"[+\.\!\/_,$%^*(+\"\']+|[+——！，〈〉《》。︱？?、．％~@#￥%……&*（）’]", "", text)
    text=text.rstrip()
    #text=' '.join(text.split())
    return text

text1 = en_df["en"].apply(clean_text)
text2 = de_df["de"].apply(clean_text)
text1 = list(text1.values)
text2 = list(text2.values)

In [ ]:
texttemp=[]
for s in text2:
    temp="cls "+s+" eos"
    texttemp.append(temp)
text2=[]
text2=texttemp
from sklearn.model_selection import train_test_split
english_words = []
french_words = []

for i in text1:
    english_words.append(len(i.split()))

for j in text2:
    french_words.append(len(j.split()))
import seaborn as sn
import matplotlib.pyplot as plt

sn.countplot(english_words)
plt.show()

sn.countplot(x = french_words)
plt.show()

In [ ]:
max_len_english = max(english_words)
max_len_french = max(french_words)

#from sklearn.model_selection import train_test_split
#x_tr,x_val,y_tr,y_val=train_test_split(text1,text2,test_size=0.3,random_state=12,shuffle=True)
#print(len(x_tr))
#print(len(x_val))
x_tr=text1[:choicenum-500]
y_tr=text2[:choicenum-500]
x_val=text1[choicenum-500:]
y_val=text2[choicenum-500:]

In [ ]:
from keras.preprocessing.text import Tokenizer
x_tokens = Tokenizer()
x_tokens.fit_on_texts(x_tr)

x_tr = x_tokens.texts_to_sequences(x_tr)
x_val = x_tokens.texts_to_sequences(x_val)

from keras.preprocessing.sequence import pad_sequences
x_tr = pad_sequences(x_tr,maxlen = max_len_english,padding = 'post')
x_val = pad_sequences(x_val,maxlen = max_len_english,padding = 'post')

# +1 for padding 
x_voc_size   =  len(x_tokens.word_index) +1

# y data
from keras.preprocessing.text import Tokenizer
y_tokens = Tokenizer()
y_tokens.fit_on_texts(y_tr)

y_tr = y_tokens.texts_to_sequences(y_tr)
y_val = y_tokens.texts_to_sequences(y_val)

from keras.preprocessing.sequence import pad_sequences
y_tr = pad_sequences(y_tr,maxlen = max_len_french,padding = 'post')
y_val = pad_sequences(y_val,maxlen = max_len_french,padding = 'post')

# +1 for padding 
y_voc_size   =  len(y_tokens.word_index) +1

In [ ]:
import pandas as pd
import pickle
import numpy as np
import os
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import Dense, GlobalAveragePooling2D,BatchNormalization,Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model
from keras.utils import np_utils
import random
from keras.preprocessing import image, sequence
import matplotlib.pyplot as plt
import keras
from keras import backend as K 
import gensim
from numpy import *
import numpy as np
import pandas as pd 
import re
from keras.applications import VGG16
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed,Conv2D,MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
modelvgg = VGG16(include_top=True,weights="imagenet")
## load the locally saved weights 
modelvgg.layers.pop()
modelvgg = Model(inputs=modelvgg.inputs, outputs=modelvgg.layers[-2].output)
modelvgg.summary()

In [ ]:
imagedata=np.zeros(shape=(choicenum,224,224,3))
for i in range(choicenum):
    temp=mp.imread(img_path[i])
    temp=cv2.resize(temp,(224,224))
    imagedata[i]=temp
imagedata=imagedata/255
imagedata=imagedata.astype(np.float16)

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from collections import OrderedDict
jpgs=img_path[:choicenum]

images = OrderedDict()
npix = 224
target_size = (npix,npix,3)
for i,name in enumerate(jpgs): 
    filename = name
    image = load_img(filename, target_size=target_size)
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    nimage = preprocess_input(image)
    y_pred = modelvgg.predict(nimage.reshape( (1,) + nimage.shape[:3]))
    images[name] = y_pred.flatten()
    if i%200==0:
        print(i,filename)

In [ ]:
vgg_imfea=np.zeros(shape=(len(jpgs),4096))
for i in range(len(jpgs)):
    vgg_imfea[i]=images[jpgs[i]]

In [ ]:
train_vggf=vgg_imfea[:choicenum-500]
val_vggf=vgg_imfea[choicenum-500:]

In [ ]:
#g_1=GlobalAveragePooling2D()(conv_3)
img_inputs=Input(shape=(4096,))
d_1=Dense(512, activation='relu')(img_inputs)
r_1=RepeatVector(max_len_english)(d_1)
vf_model = Model(img_inputs, r_1)
vf_model.summary()

In [ ]:
x_voc=x_voc_size
y_voc=y_voc_size

# img_inputs=Input(shape=(224,224,3))
# conv_1=Conv2D(filters=64, kernel_size=(3,3), strides=(1, 1), padding='valid')(img_inputs)
# m_pool=MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')(conv_1)
# bn_1=BatchNormalization()(m_pool)
# conv_2=Conv2D(filters=128, kernel_size=(3,3), strides=(1, 1), padding='valid')(bn_1)
# conv_2=Conv2D(filters=128, kernel_size=(3,3), strides=(1, 1), padding='valid')(conv_2)
# m_pool1=MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')(conv_2)
# conv_3=Conv2D(filters=256, kernel_size=(3,3), strides=(1, 1), padding='valid')(m_pool1)
# conv_3=Conv2D(filters=256, kernel_size=(3,3), strides=(1, 1), padding='valid')(conv_3)
# m_pool2=MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')(conv_3)
# bn_2=BatchNormalization()(m_pool2)
# conv_3=Conv2D(filters=512, kernel_size=(3,3), strides=(1, 1), padding='valid')(bn_2)
# conv_3=Conv2D(filters=512, kernel_size=(3,3), strides=(1, 1), padding='valid')(conv_3)
# g_1=GlobalAveragePooling2D()(conv_3)
# d_1=Dense(512, activation='relu')(g_1)
# r_1=RepeatVector(max_len_english)(d_1)
# vf_model = Model(img_inputs, r_1)
#vf_model.summary()


latent_dim = 512
embedding_dim=512

# Encoder
encoder_inputs = Input(shape=(max_len_english,))

#embedding layer
enc_emb =  Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

encoder_output2=Concatenate(axis=-1)([encoder_output2,r_1])

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model 
#model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model = Model([encoder_inputs,decoder_inputs,img_inputs], decoder_outputs)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
history=model.fit([x_tr,y_tr[:,:-1],train_vggf], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,validation_data=([x_val,y_val[:,:-1],val_vggf], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]),epochs=100,batch_size=512)

In [ ]:
#history=model.fit([x_tr,y_tr[:,:-1],vgg_imfea], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=40,batch_size=512)

In [ ]:
reverse_target_word_index=y_tokens.index_word
reverse_source_word_index=x_tokens.index_word
target_word_index=y_tokens.word_index

# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=[encoder_inputs,img_inputs],outputs=[encoder_outputs, state_h, state_c])
encoder_model.summary()

In [ ]:

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_len_english,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])
decoder_model.summary()

In [ ]:
def decode_sequence(input_seq,img):
    img=img[np.newaxis,:]
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict([input_seq,img])
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['cls']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eos'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eos'  or len(decoded_sentence.split()) >= (max_len_french -1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['cls']) and i!=target_word_index['eos']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

for i in range(5):
    print("Review:",seq2text(x_tr[i]))
    print("Original summary:",seq2summary(y_tr[i]))
    print("Predicted summary:",decode_sequence(x_tr[i].reshape(1,max_len_english),train_vggf[i]))
    print("\n")

In [ ]:
i=3
print("Review:",seq2text(x_tr[i]))
print("Original summary:",seq2summary(y_tr[i]))
print("Predicted summary:",decode_sequence(x_tr[i].reshape(1,max_len_english),vgg_imfea[i]))
plt.imshow(imagedata[i].astype(np.float32))

In [ ]:
i=2
print("Review:",seq2text(x_tr[i]))
print("Original summary:",seq2summary(y_tr[i]))
print("Predicted summary:",decode_sequence(x_tr[i].reshape(1,max_len_english),vgg_imfea[i]))
plt.imshow(imagedata[i].astype(np.float32))

In [ ]:
i=0
print("Review:",seq2text(x_tr[i]))
print("Original summary:",seq2summary(y_tr[i]))
print("Predicted summary:",decode_sequence(x_tr[i].reshape(1,max_len_english),vgg_imfea[i]))
plt.imshow(imagedata[i].astype(np.float32))

In [ ]:
!pip install sacrebleu
import sacrebleu
import random

In [ ]:
temp_o=[]
temp_p=[]
for i in range(50):
    s=random.randint(0,len(y_tr)-1)
    temp_o.append(seq2summary(y_tr[s]))
    temp_p.append(decode_sequence(x_tr[s].reshape(1,max_len_english),train_vggf[s]))

bleu = sacrebleu.corpus_bleu(temp_o, [temp_p],lowercase=True, tokenize='intl')
print(bleu.score)

In [ ]:
temp_o=[]
temp_p=[]
for i in range(100):
    s=random.randint(0,len(y_tr)-1)
    temp_o.append(seq2summary(y_tr[s]))
    temp_p.append(decode_sequence(x_tr[s].reshape(1,max_len_english),train_vggf[s]))

bleu = sacrebleu.corpus_bleu(temp_o, [temp_p],lowercase=True, tokenize='intl')
print(bleu.score)

In [ ]:
temp_o=[]
temp_p=[]
for i in range(100):
    s=random.randint(0,len(y_tr)-1)
    temp_o.append(seq2summary(y_tr[s]))
    temp_p.append(decode_sequence(x_tr[s].reshape(1,max_len_english),val_vggf[s]))

bleu = sacrebleu.corpus_bleu(temp_o, [temp_p],lowercase=True, tokenize='intl')
print(bleu.score)